In [1]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Cargamos el archivo csv
df=pd.read_csv('Datos_limpios_Los_Angeles_Estados_Unidos.csv')
#Eliminamos la columna unnamed
df= df.drop(['Unnamed: 0'], axis=1)
df.head(20)

,last_scraped,source,description,neighborhood_overview,host_name,host_since,host_response_time,host_is_superhost,host_neighbourhood,host_verifications,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,45540,city scrape,The whole group will enjoy easy access to ever...,No overview available,Moon,41294.0,within a few hours,f,Brentwood,"['email', 'phone']",...,4.90,5.00,5.00,4.90,4.80,3.9,4.0,0.2,0.0,0.80
1,45540,previous scrape,No description available,No overview available,Ida,43668.0,Unknown,f,South Robertson,"['email', 'phone']",...,4.90,5.00,5.00,4.90,4.80,1.0,1.0,0.0,0.0,0.80
2,45540,city scrape,"Charming, renovated home in the Beverly Hills ...",No overview available,Tiana,45054.0,within a day,f,Unknown,"['email', 'phone']",...,4.90,5.00,5.00,4.90,4.80,4.0,4.0,0.0,0.0,0.80
3,45539,city scrape,Relax in this unique and serene retreat.,No overview available,Dan,44522.0,a few days or more,f,Walnut Valley,['phone'],...,5.00,5.00,5.00,5.00,4.80,4.0,1.0,0.2,0.0,0.06
4,45540,city scrape,Welcome to our stunning apartment nestled in t...,No overview available,Farkhat,45136.0,within an hour,t,Downtown,['phone'],...,4.98,4.93,5.00,4.73,4.77,3.0,3.0,0.0,0.0,3.41
5,45540,city scrape,This room comes fully furnished and cannot be ...,No overview available,Christos,45313.0,within a day,f,Central LA,"['email', 'phone']",...,4.90,5.00,5.00,4.90,4.80,15.0,0.0,0.2,0.0,0.80
6,45540,city scrape,No description available,"Eclectic nice neighborhood, beautiful weather.",Sara,41227.0,within a few hours,f,Del Rey,"['email', 'phone']",...,4.90,4.92,4.85,4.76,4.65,2.0,2.0,0.0,0.0,3.21
7,45540,city scrape,Escape to a luxurious retreat on the historic ...,No overview available,Tig,45035.0,within an hour,t,Venice,['phone'],...,5.00,4.80,4.97,4.94,4.80,1.0,1.0,0.0,0.0,2.34
8,45540,city scrape,Hosted by @The_BonaBona - Stunning resort styl...,No overview available,Bona Bona,44376.0,within an hour,t,North Hollywood,"['email', 'phone']",...,5.00,5.00,5.00,5.00,4.90,21.0,2.1,0.0,0.0,1.23
9,45540,city scrape,You'll have a great time at this comfortable p...,No overview available,Max,44866.0,a few days or more,f,Westlake,"['email', 'phone']",...,4.90,5.00,5.00,4.90,4.80,2.0,2.0,0.0,0.0,0.80


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45533 entries, 0 to 45532
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  45533 non-null  int64  
 1   source                                        45533 non-null  object 
 2   description                                   45533 non-null  object 
 3   neighborhood_overview                         45533 non-null  object 
 4   host_name                                     45533 non-null  object 
 5   host_since                                    45533 non-null  float64
 6   host_response_time                            45533 non-null  object 
 7   host_is_superhost                             45533 non-null  object 
 8   host_neighbourhood                            45533 non-null  object 
 9   host_verifications                            45533 non-null 

In [4]:
#Verificar los valores sin repetirse de una columna
unico1 = np.unique(df['room_type'])
unico1

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [5]:
#Verificar los valores sin repetirse de una columna
unico2 = np.unique(df['bathrooms_text'])
unico2

array(['0 baths', '0 shared baths', '1 bath', '1 private bath',
       '1 shared bath', '1.5 baths', '1.5 shared baths', '10 baths',
       '10 shared baths', '10.5 baths', '11 baths', '11 shared baths',
       '11.5 baths', '11.5 shared baths', '12 baths', '12.5 baths',
       '13 baths', '13 shared baths', '13.5 baths', '16 baths', '2 baths',
       '2 shared baths', '2.5 baths', '2.5 shared baths', '21 baths',
       '24 baths', '3 baths', '3 shared baths', '3.5 baths',
       '3.5 shared baths', '4 baths', '4 shared baths', '4.5 baths',
       '4.5 shared baths', '5 baths', '5 shared baths', '5.5 baths',
       '5.5 shared baths', '50 baths', '6 baths', '6 shared baths',
       '6.5 baths', '7 baths', '7 shared baths', '7.5 baths', '8 baths',
       '8 shared baths', '8.5 baths', '8.5 shared baths', '9 baths',
       '9.5 baths', 'Half-bath', 'Private half-bath', 'Shared half-bath'],
      dtype=object)

In [6]:
#Convierto una variable a dicotómica
df['room_type']= df['room_type'].replace(["Hotel room", "Private room", "Shared room"], "No Entire home/apt")
df['room_type']

0           Entire home/apt
1           Entire home/apt
2           Entire home/apt
3        No Entire home/apt
4           Entire home/apt
                ...        
45528       Entire home/apt
45529       Entire home/apt
45530    No Entire home/apt
45531       Entire home/apt
45532       Entire home/apt
Name: room_type, Length: 45533, dtype: object

In [7]:
# Extraer el número de baños como flotante
df["bathrooms_num"] = df["bathrooms_text"].str.extract(r'(\d+\.?\d*)').astype(float)

# Crear la variable dicotómica de manera automática
df["bathrooms_text"] = np.where(df["bathrooms_num"] > 1, ">1 baño", "≤1 baño")

# Verificar los cambios
df["bathrooms_text"].value_counts()


bathrooms_text
≤1 baño    28368
>1 baño    17165
Name: count, dtype: int64

In [8]:
#Seleccion de variables dependientes
Var_Dep1 = df["host_is_superhost"].unique()
print(Var_Dep1)
Var_Dep2 = df["instant_bookable"].unique()
print(Var_Dep2)
Var_Dep3 = df["host_identity_verified"].unique()
print(Var_Dep3)
Var_Dep4 = df["room_type"].unique()
print(Var_Dep4)
Var_Dep5 = df["bathrooms_text"].unique()
print(Var_Dep5)


['f' 't']
['f' 't']
['t' 'f']
['Entire home/apt' 'No Entire home/apt']
['>1 baño' '≤1 baño']


**Primer modelo de regresion logica.**

In [ ]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep= df[['price']]
Var_Dep1= df['host_is_superhost']

#Redefinimos las variables
X= Vars_Indep
y= Var_Dep1

In [25]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [26]:
#Se escalan todos los datos
escalar = StandardScaler()

In [27]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [28]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [29]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [30]:
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(13660,), dtype=object)

In [31]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confunsion 1:')
print(matriz)

Matriz de Confunsion 1:
[[8548    0]
 [5112    0]]


In [33]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precision del modelo 1:')
print(precision)

Precision del modelo 1:
0.6257686676427525


In [35]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo 1:')
print(exactitud)

Exactitud del modelo 1:
0.6257686676427525


In [39]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo 1:')
print(sensibilidad)

Sensibilidad del modelo 1:
0.0


**Segundo modelo de regresion logistica.**

In [ ]:
df.info()

In [41]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep2= df[['review_scores_rating', 'review_scores_communication', 'review_scores_checkin']]
Var_Dep2= df['host_is_superhost']

#Redefinimos las variables
X= Vars_Indep2
y= Var_Dep2

In [43]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [44]:
#Realizamos una prediccion
y_pred2 = algoritmo.predict(X_test)
y_pred2

array(['f', 'f', 't', ..., 'f', 'f', 'f'], shape=(13660,), dtype=object)

In [55]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred2)
print('Matriz de Confunsion 2:')
print(matriz)

Matriz de Confunsion 2:
[[8697 1443]
 [3026  494]]


In [56]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred2, average="binary", pos_label="t")
print('Precision del modelo 2:')
print(precision)

Precision del modelo 2:
0.2550335570469799


In [57]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred2)
print('Exactitud del modelo 2:')
print(exactitud)

Exactitud del modelo 2:
0.6728404099560761


In [58]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred2, average="binary", pos_label="f")
print('Sensibilidad del modelo 2:')
print(sensibilidad)

Sensibilidad del modelo 2:
0.8576923076923076


**Tercer modelo de regresion logistica.**

In [ ]:
df.info()

In [52]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep3= df[['review_scores_rating', 'price', 'host_acceptance_rate']]
Var_Dep3= df['instant_bookable']

#Redefinimos las variables
X= Vars_Indep3
y= Var_Dep3

In [53]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [54]:
#Realizamos una prediccion
y_pred3 = algoritmo.predict(X_test)
y_pred3

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(13660,), dtype=object)

In [59]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred3)
print('Matriz de Confunsion 3:')
print(matriz)

Matriz de Confunsion 3:
[[10113    27]
 [ 3481    39]]


In [60]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred3, average="binary", pos_label="t")
print('Precision del modelo 3:')
print(precision)

Precision del modelo 3:
0.5909090909090909


In [61]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred3)
print('Exactitud del modelo 3:')
print(exactitud)

Exactitud del modelo 3:
0.7431918008784774


In [62]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred3, average="binary", pos_label="f")
print('Sensibilidad del modelo 3:')
print(sensibilidad)

Sensibilidad del modelo 3:
0.9973372781065089


**Cuarto modelo de regresion logistica.**

In [67]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep4= df[['number_of_reviews', 'price']]
Var_Dep4= df['instant_bookable']

#Redefinimos las variables
X= Vars_Indep4
y= Var_Dep4

In [68]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [69]:
#Realizamos una prediccion
y_pred4 = algoritmo.predict(X_test)
y_pred4

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(13660,), dtype=object)

In [70]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred4)
print('Matriz de Confunsion 4:')
print(matriz)

Matriz de Confunsion 4:
[[10195     0]
 [ 3465     0]]


In [72]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred4, average="binary", pos_label="f")
print('Precision del modelo 4:')
print(precision)

Precision del modelo 4:
0.7463396778916545


In [73]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred4)
print('Exactitud del modelo 4:')
print(exactitud)

Exactitud del modelo 4:
0.7463396778916545


In [75]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred4, average="binary", pos_label="t")
print('Sensibilidad del modelo 4:')
print(sensibilidad)

Sensibilidad del modelo 4:
0.0


**Quinto modelo de regresion logistica.**

In [76]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep5= df[['number_of_reviews', 'price', 'host_total_listings_count']]
Var_Dep5= df['host_identity_verified']

#Redefinimos las variables
X= Vars_Indep5
y= Var_Dep5

In [77]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [78]:
#Realizamos una prediccion
y_pred5 = algoritmo.predict(X_test)
y_pred5

array(['t', 't', 't', ..., 't', 't', 't'], shape=(13660,), dtype=object)

In [79]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred5)
print('Matriz de Confunsion 5:')
print(matriz)

Matriz de Confunsion 5:
[[    0  1465]
 [    0 12195]]


In [80]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred5, average="binary", pos_label="t")
print('Precision del modelo 5:')
print(precision)

Precision del modelo 5:
0.8927525622254758


In [81]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred5)
print('Exactitud del modelo 5:')
print(exactitud)

Exactitud del modelo 5:
0.8927525622254758


In [82]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred5, average="binary", pos_label="f")
print('Sensibilidad del modelo 5:')
print(sensibilidad)

Sensibilidad del modelo 5:
0.0


**Sexto modelo de regresion logistica.**

In [ ]:
df.info()

In [84]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep6= df[['host_acceptance_rate', 'review_scores_rating']]
Var_Dep6= df['host_identity_verified']

#Redefinimos las variables
X= Vars_Indep6
y= Var_Dep6

In [85]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [86]:
#Realizamos una prediccion
y_pred6 = algoritmo.predict(X_test)
y_pred6

array(['t', 't', 't', ..., 't', 't', 't'], shape=(13660,), dtype=object)

In [87]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred6)
print('Matriz de Confunsion 6:')
print(matriz)

Matriz de Confunsion 6:
[[    0  1443]
 [    0 12217]]


In [88]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred6, average="binary", pos_label="t")
print('Precision del modelo 6:')
print(precision)

Precision del modelo 6:
0.8943631039531479


In [89]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred6)
print('Exactitud del modelo 6:')
print(exactitud)

Exactitud del modelo 6:
0.8943631039531479


In [90]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred6, average="binary", pos_label="f")
print('Sensibilidad del modelo 6:')
print(sensibilidad)

Sensibilidad del modelo 6:
0.0


**Septimo modelo de regresion logistica.**

In [ ]:
df.info()

In [92]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep7= df[['accommodates', 'host_acceptance_rate', 'price']]
Var_Dep7= df['room_type']

#Redefinimos las variables
X= Vars_Indep7
y= Var_Dep7

In [93]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [94]:
#Realizamos una prediccion
y_pred7 = algoritmo.predict(X_test)
y_pred7

array(['Entire home/apt', 'Entire home/apt', 'No Entire home/apt', ...,
       'Entire home/apt', 'No Entire home/apt', 'Entire home/apt'],
      shape=(13660,), dtype=object)

In [95]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred7)
print('Matriz de Confunsion 7:')
print(matriz)

Matriz de Confunsion 7:
[[9388  669]
 [1884 1719]]


In [104]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred7, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo 7:')
print(precision)

Precision del modelo 7:
0.7198492462311558


In [105]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred7)
print('Exactitud del modelo 7:')
print(exactitud)

Exactitud del modelo 7:
0.813103953147877


In [106]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred7, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo 7:')
print(sensibilidad)

Sensibilidad del modelo 7:
0.9334791687381923


**Octavo modelo de regresion logistica.**

In [ ]:
df.info()

In [108]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep8= df[['number_of_reviews', 'bedrooms']]
Var_Dep8= df['room_type']

#Redefinimos las variables
X= Vars_Indep8
y= Var_Dep8

In [109]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [110]:
#Realizamos una prediccion
y_pred8 = algoritmo.predict(X_test)
y_pred8

array(['Entire home/apt', 'Entire home/apt', 'Entire home/apt', ...,
       'Entire home/apt', 'Entire home/apt', 'Entire home/apt'],
      shape=(13660,), dtype=object)

In [111]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred8)
print('Matriz de Confunsion 8:')
print(matriz)

Matriz de Confunsion 8:
[[9870  235]
 [3533   22]]


In [119]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred8, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo 8:')
print(precision)

Precision del modelo 8:
0.08560311284046693


In [120]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred8)
print('Exactitud del modelo 8:')
print(exactitud)

Exactitud del modelo 8:
0.7241581259150806


In [118]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred8, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo 8:')
print(sensibilidad)

Sensibilidad del modelo 8:
0.9767441860465116


**Noveno modelo de regresion logistica.**

In [121]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep9= df[['number_of_reviews', 'price', 'accommodates']]
Var_Dep9= df['bathrooms_text']

#Redefinimos las variables
X= Vars_Indep9
y= Var_Dep9

In [122]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [123]:
#Realizamos una prediccion
y_pred9 = algoritmo.predict(X_test)
y_pred9

array(['>1 baño', '≤1 baño', '>1 baño', ..., '≤1 baño', '≤1 baño',
       '≤1 baño'], shape=(13660,), dtype=object)

In [124]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred9)
print('Matriz de Confunsion 9:')
print(matriz)

Matriz de Confunsion 9:
[[3308 1759]
 [ 654 7939]]


In [125]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred9, average="binary", pos_label=">1 baño")
print('Precision del modelo 9:')
print(precision)

Precision del modelo 9:
0.8349318525996972


In [126]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred9)
print('Exactitud del modelo 9:')
print(exactitud)

Exactitud del modelo 9:
0.8233528550512446


In [127]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred9, average="binary", pos_label="≤1 baño")
print('Sensibilidad del modelo 9:')
print(sensibilidad)

Sensibilidad del modelo 9:
0.9238915396252764


**Decimo modelo de regresion logistica.**

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45533 entries, 0 to 45532
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  45533 non-null  int64  
 1   source                                        45533 non-null  object 
 2   description                                   45533 non-null  object 
 3   neighborhood_overview                         45533 non-null  object 
 4   host_name                                     45533 non-null  object 
 5   host_since                                    45533 non-null  float64
 6   host_response_time                            45533 non-null  object 
 7   host_is_superhost                             45533 non-null  object 
 8   host_neighbourhood                            45533 non-null  object 
 9   host_verifications                            45533 non-null 

In [129]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep10= df[['host_acceptance_rate', 'review_scores_rating']]
Var_Dep10= df['bathrooms_text']

#Redefinimos las variables
X= Vars_Indep10
y= Var_Dep10

In [130]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [131]:
#Realizamos una prediccion
y_pred10 = algoritmo.predict(X_test)
y_pred10

array(['≤1 baño', '≤1 baño', '≤1 baño', ..., '≤1 baño', '≤1 baño',
       '≤1 baño'], shape=(13660,), dtype=object)

In [132]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred10)
print('Matriz de Confunsion 10:')
print(matriz)

Matriz de Confunsion 10:
[[   0 5234]
 [   0 8426]]


In [134]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred10, average="binary", pos_label="≤1 baño")
print('Precision del modelo 10:')
print(precision)

Precision del modelo 10:
0.6168374816983895


In [135]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred10)
print('Exactitud del modelo 10:')
print(exactitud)

Exactitud del modelo 10:
0.6168374816983895


In [136]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred10, average="binary", pos_label=">1 baño")
print('Sensibilidad del modelo 10:')
print(sensibilidad)

Sensibilidad del modelo 10:
0.0
